In [1]:
%pwd
%cd ..
%pwd

/home/grkmkola/Desktop/Projects/mlops-proje/kidney-disease-classification


/home/grkmkola/miniconda3/envs/kidney/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/home/grkmkola/Desktop/Projects/mlops-proje/kidney-disease-classification'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_include_top: bool
    params_learning_rate: float
    params_weights: str
    params_classes: int

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
        ) -> None:
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories(
            [
                self.config.artifacts_root,
                self.config.prepare_base_model.root_dir
            ]
        )

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [13]:
import torch
import torch.nn as nn
from torchvision.models import resnet50
from torchsummary import summary
from pathlib import Path

In [22]:
class PrepareBaseModel:
    def __init__(self, config):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    def get_base_model(self):
        self.model = resnet50(weights=self.config.params_weights)
        if not self.config.params_include_top:
            self.model = nn.Sequential(*list(self.model.children())[:-1])
        self.model = self.model.to(self.device)
        self.save_model(path=self.config.base_model_path, model=self.model)

    def _prepare_full_model(self, model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for param in model.parameters():
                param.requires_grad = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for param in list(model.parameters())[:-freeze_till]:
                param.requires_grad = False

        if self.config.params_include_top:
            model.fc = nn.Linear(model.fc.in_features, classes)
        else:
            model.add_module('flatten', nn.Flatten())
            model.add_module('fc', nn.Linear(2048, classes))  # 2048 is the output size of ResNet50's last conv layer

        model = model.to(self.device)

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

        return model, criterion, optimizer
    
    def update_base_model(self):
        self.full_model, self.criterion, self.optimizer = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: nn.Module):
        torch.save(model.state_dict(), path)

    def load_model(self, path: Path):
        self.model.load_state_dict(torch.load(path))
        self.model.to(self.device)

    def summary(self):
        if self.full_model is None:
            raise ValueError("Model hasn't been prepared yet. Call update_base_model() first.")
        
        # Adjusting input_size format for torchsummary
        # Assuming self.config.IMAGE_SIZE is in the format [height, width, channels]
        input_size = (self.config.params_image_size[2], self.config.params_image_size[0], self.config.params_image_size[1])
        
        summary(self.full_model, input_size=input_size, device=str(self.device))
    

In [25]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
    prepare_base_model.summary()
except Exception as e:
    raise e

[2024-07-22 14:48:34,994: INFO: utils: yaml file config/config.yaml loaded successfully:]
[2024-07-22 14:48:34,997: INFO: utils: yaml file params.yaml loaded successfully:]
[2024-07-22 14:48:34,998: INFO: utils: created directory at: artifacts:]
[2024-07-22 14:48:34,999: INFO: utils: created directory at: artifacts/prepare_base_model:]
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9 